<a href="https://colab.research.google.com/github/prishaa09/Solaris/blob/main/Solar_Flare_Phase_2_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q "sunpy[all]" zeep drms

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.4/966.4 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.0/98.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.8/96.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 350.3/350.3 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
from sunpy.net import Fido, attrs as a
from sunpy import timeseries as ts

# --------------- CONFIG ----------------
START = "2015-01-01 00:00"     # start of interval
END   = "2015-03-31 23:59"     # end of interval
OUT_DIR = Path("/content/data_goes")   # where CSV will be saved
CSV_NAME = f"goes_xrs_{START[:10]}_to_{END[:10]}_1min_basic.csv"
CADENCE = "1min"
MIN_ALLOWED = 0.0              # drop non-physical negatives
# --------------------------------------


In [13]:
def fetch_goes_xrs(start: str, end: str):
    """
    Fetch GOES XRS data and ALWAYS return ONE merged TimeSeries.
    Compatible with SunPy 5.1 (ts.TimeSeries returns a list-like object).
    """
    res = Fido.search(a.Time(start, end), a.Instrument("XRS"))
    files = Fido.fetch(res)

    # Load data – can be a single TS or a list-like container
    ts_obj = ts.TimeSeries(files)

    # Case 1: Already a single TimeSeries
    from sunpy.timeseries.timeseriesbase import GenericTimeSeries
    if isinstance(ts_obj, GenericTimeSeries):
        return ts_obj

    # Case 2: A list of TimeSeries objects → merge manually
    # ts_obj is iterable even if not a list, so safe to iterate
    dfs = []
    for part in ts_obj:
        dfs.append(part.to_dataframe())

    merged_df = pd.concat(dfs)
    merged_df = merged_df.sort_index()
    merged_df = merged_df[~merged_df.index.duplicated(keep="last")]

    return GenericTimeSeries(merged_df)

def clean_goes_basic(goes_ts, cadence="1min"):
    df = goes_ts.to_dataframe()

    # 🟦 1. Force clean DatetimeIndex
    df.index = pd.to_datetime(df.index, errors="coerce")
    df = df[df.index.notna()]
    df.index = df.index.astype("datetime64[ns]")

    # 🟦 2. Sort + collapse duplicates (even invisible ones)
    df = df.sort_index()
    df = df.groupby(level=0).mean()

    # 🟦 3. Standardize names
    rename_map = {}
    for c in df.columns:
        lc = c.lower()
        if "xrsb" in lc or "1-8" in lc:
            rename_map[c] = "flux_1_8A"
        if "xrsa" in lc or "0.5-4" in lc:
            rename_map[c] = "flux_0.5_4A"
        if "quality" in lc:
            rename_map[c] = "quality_flag"
    df = df.rename(columns=rename_map)

    # Remove duplicate cols after renaming
    df = df.loc[:, ~df.columns.duplicated(keep="last")]

    # 🟦 4. Filter good/unknown quality
    if "quality_flag" in df.columns:
        df = df[df["quality_flag"].isna() | (df["quality_flag"] == 0)]

    # 🟦 5. NOW resample safely
    df = df.resample(cadence).mean()

    # 🟦 6. Derived ratio
    if {"flux_1_8A", "flux_0.5_4A"}.issubset(df.columns):
        df["ratio"] = df["flux_0.5_4A"] / df["flux_1_8A"]

    # 🟦 7. Drop empty rows
    df = df.dropna(subset=["flux_1_8A", "flux_0.5_4A"], how="all")

    return df.reset_index().rename(columns={"index": "time_utc"})

def save_csv(df: pd.DataFrame, out_dir: Path, name: str) -> Path:
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / name
    df.to_csv(out_path, index=False)
    print(f"✅ Saved: {out_path}")
    return out_path


In [14]:
print(f"Fetching GOES XRS from {START} to {END} …")
goes_ts = fetch_goes_xrs(START, END)
df = clean_goes_basic(goes_ts, cadence=CADENCE)
csv_path = save_csv(df, OUT_DIR, CSV_NAME)

print("Shape:", df.shape)
print(df.head())

# See file on the Colab VM
!ls -lh /content/data_goes

# Download the CSV to your laptop
from google.colab import files
files.download(str(csv_path))


Fetching GOES XRS from 2015-01-01 00:00 to 2015-03-31 23:59 …


Files Downloaded:   0%|          | 0/332 [00:00<?, ?file/s]

✅ Saved: /content/data_goes/goes_xrs_2015-01-01_to_2015-03-31_1min_basic.csv
Shape: (126729, 5)
             time_utc   flux_0.5_4A  flux_1_8A  quality_flag     ratio
0 2015-01-01 00:00:00  2.566501e-08   0.000001           0.0  0.018824
1 2015-01-01 00:01:00  2.498404e-08   0.000001           0.0  0.018463
2 2015-01-01 00:02:00  2.319489e-08   0.000001           0.0  0.017297
3 2015-01-01 00:03:00  2.448375e-08   0.000001           0.0  0.018494
4 2015-01-01 00:04:00  2.148839e-08   0.000001           0.0  0.016373
total 7.8M
-rw-r--r-- 1 root root 7.7M Nov 27 18:46 goes_xrs_2015-01-01_to_2015-03-31_1min_basic.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>